In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import sklearn as sk
from sklearn import decomposition as dec
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.utils.prune as prune

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [43]:
path = './cifar_net.pt'
model = nn.Module()
model = torch.load(path)

model.eval()

NN(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

 Eliminating negative values within the weight matrix

In [12]:
#pruning 

prune.random_unstructured(model.fc1, name='weight', amount=0.5)
print(model.fc1._forward_pre_hooks)


OrderedDict([(1, <torch.nn.utils.prune.RandomUnstructured object at 0x000001983DC37D30>)])


In [13]:
print(model.fc1.weight)

tensor([[ 0.1947,  0.0000,  0.0000,  ...,  0.0300,  0.0916, -0.0000],
        [ 0.2070, -0.0510, -0.0575,  ...,  0.0000,  0.1574, -0.0447],
        [-0.0000, -0.1813, -0.0000,  ..., -0.0422, -0.0474,  0.0000],
        ...,
        [-0.0000, -0.0000,  0.0016,  ...,  0.0000,  0.0000,  0.0881],
        [ 0.0132, -0.0325, -0.0655,  ...,  0.0000, -0.0380,  0.0782],
        [-0.2341, -0.3235, -0.0489,  ...,  0.0000,  0.1251,  0.0000]],
       grad_fn=<MulBackward0>)


In [39]:
datasample = iter(testloader)
img, lbl = datasample.next()
output = model(img)

print(output)
acc, pred = torch.max(output, dim=1)
print(pred)

tensor([[-0.1597, -0.7224, -0.4315,  0.6580,  0.6771,  0.1744, -0.0459, -0.1980,
          0.1730, -0.0792],
        [-0.1597, -0.7224, -0.4315,  0.6580,  0.6771,  0.1744, -0.0459, -0.1980,
          0.1730, -0.0792],
        [-0.1597, -0.7224, -0.4315,  0.6580,  0.6771,  0.1744, -0.0459, -0.1980,
          0.1730, -0.0792],
        [-0.1597, -0.7224, -0.4315,  0.6580,  0.6771,  0.1744, -0.0459, -0.1980,
          0.1730, -0.0792]], grad_fn=<AddmmBackward0>)
tensor([4, 4, 4, 4])


In [80]:
fc_1 = model.fc1.weight.detach().numpy().T


new_nmf = dec.NMF(n_components=60, init='random', random_state=42)
fc_1_reduced = new_nmf.fit_transform(fc_1)


done!!


In [40]:
from torch.profiler import profile, record_function, ProfilerActivity
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

print(optimizer)

running_loss = 0.0
total = 0.0
correct = 0.0
loss_nodr =[]
acc_nodr = []
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

for epoch in range(5):
    
    print('epoch:', epoch)
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad(True)
        with torch.set_grad_enabled(True):
            with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
             outputs = model(inputs).to(device)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        running_loss += loss.item()
        if i % 2000 == 1999:
            loss_temp = running_loss / 2000
            acc_temp = 100 * correct / total
            loss_nodr.append(loss_temp)
            acc_nodr.append(acc_temp)
            print(
                f'[{epoch + 1}, {i + 1:5d}] loss: {loss_temp:.3f} acc: {acc_temp:.3f}')
            running_loss = 0.0
            correct = 0.0
            total = 0.0
print('Finished Training')


SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0


c:\Users\sulta\anaconda3\lib\site-packages\torch\autograd\profiler.py:160: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")


[1,  2000] loss: 2.323 acc: 9.925
[1,  4000] loss: 2.304 acc: 9.850
[1,  6000] loss: 2.303 acc: 10.037
[1,  8000] loss: 2.303 acc: 10.137
[1, 10000] loss: 2.303 acc: 9.700
[1, 12000] loss: 2.303 acc: 10.438
epoch: 1
[2,  2000] loss: 2.879 acc: 9.820
[2,  4000] loss: 2.303 acc: 9.863
[2,  6000] loss: 2.303 acc: 9.938
[2,  8000] loss: 2.303 acc: 9.963
[2, 10000] loss: 2.303 acc: 9.637
[2, 12000] loss: 2.303 acc: 10.213
epoch: 2
[3,  2000] loss: 2.879 acc: 9.830
[3,  4000] loss: 2.303 acc: 9.575
[3,  6000] loss: 2.303 acc: 10.300
[3,  8000] loss: 2.303 acc: 10.537
[3, 10000] loss: 2.303 acc: 9.562
[3, 12000] loss: 2.303 acc: 9.688
epoch: 3
[4,  2000] loss: 2.879 acc: 9.730
[4,  4000] loss: 2.303 acc: 10.525
[4,  6000] loss: 2.303 acc: 9.588
[4,  8000] loss: 2.303 acc: 9.637
[4, 10000] loss: 2.303 acc: 9.875
[4, 12000] loss: 2.303 acc: 9.725
epoch: 4
[5,  2000] loss: 2.879 acc: 9.800
[5,  4000] loss: 2.303 acc: 9.912
[5,  6000] loss: 2.303 acc: 10.088
[5,  8000] loss: 2.303 acc: 10.075
[5,